In [1]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import cv2
import random
from config.coco_config import config

In [2]:
config=config()

In [3]:
resnet50 = tf.keras.applications.ResNet50(input_shape=[config.input_shape[0], config.input_shape[1], 3],
                                               include_top=False,
                                               weights='imagenet')

In [4]:
resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 192, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 198, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 96, 64)  9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 96, 64)  256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [14]:
inputs = tf.keras.Input(shape=(config.input_shape[0], config.input_shape[1], 3))

#Base network from Resnet50
features = resnet50(inputs)


#Deconvolution Head Network
#1
features = tf.keras.layers.Conv2DTranspose(256, [4, 4], strides=2,
                                    padding='same',
                                    kernel_initializer=tf.random_normal_initializer(0., 0.01))(features)
features = tf.keras.layers.BatchNormalization()(features)
features = tf.keras.layers.ReLU()(features)
#2
features = tf.keras.layers.Conv2DTranspose(256, [4, 4], strides=2,
                                    padding='same',
                                    kernel_initializer=tf.random_normal_initializer(0., 0.01))(features)
features = tf.keras.layers.BatchNormalization()(features)
features = tf.keras.layers.ReLU()(features)
#3
features = tf.keras.layers.Conv2DTranspose(256, [4, 4], strides=2,
                                    padding='same',
                                    kernel_initializer=tf.random_normal_initializer(0., 0.01))(features)
features = tf.keras.layers.BatchNormalization()(features)
features = tf.keras.layers.ReLU()(features)

out = tf.keras.layers.Conv2D(config.num_kps, (1, 1), 
                             padding='same', 
                             kernel_initializer=tf.keras.initializers.VarianceScaling())(features)

model = tf.keras.Model(inputs, out)

In [15]:
model.save('model/pose.h5')